# Misc

In [ ]:
# changing theme
# !jt -t chesterish 
# !jt -t onedork -fs 100 -altp -tfs 14 -nfs 115 -T
# restore default theme
!jt -r

# Comparing fake and real faces

In [ ]:
import sys 
sys.path.append(r'C:\Users\Jonas\PycharmProjects\IlkinThesis\ThesisExperiments')
from IPython.display import Image, Video

import cv2
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import AgglomerativeClustering

import config
import cleaning_with_face_recognition as rec

In [ ]:
# finds multiple face folders
folders = glob.glob(config.FACE_IMAGES + '/*')
metadata = pd.read_json(config.METADATA_PATH).T

multiple_face_videos = []

for f in folders:
    imgs = glob.glob(f + '/*')
    if len(imgs) < 30:
        continue
    if metadata.loc[os.path.basename(f)]['label'] == 'REAL':
        continue
    org = metadata.loc[os.path.basename(f)]['original']
    print(f, len(imgs), org)
    multiple_face_videos.append((f, org))
    fake_path = f
    real_path = os.path.join(config.FACE_IMAGES, metadata.loc[os.path.basename(f)]['original'])
    


## Helper classes & functions

In [ ]:
class Face:
    def __init__(self, path):
        self.face = cv2.imread(path)
    
    def __sub__(self, other):
        diff = abs(self.face.astype('int') - other.face.astype('int'))
        return diff
    
    def get(self, type='rgb'):
        if type == 'bgr':
            return self.face
        elif type == 'rgb':
            return cv2.cvtColor(self.face, cv2.COLOR_BGR2RGB)
        elif type == 'gray':
            return cv2.cvtColor(self.face, cv2.COLOR_BGR2GRAY)

In [ ]:
class Loader:
    def __init__(self, path):
        self.path = path
        self.names = glob.glob(path + '/*')
        
    def __getitem__(self, idx):
        return self.names[idx]
    
    def __len__(self):
        return len(self.names)

In [ ]:
class Metadata:
    def __init__(self, path):
        self.metadata = pd.read_json(config.METADATA_PATH).T
    
    def __getitem__(self, name):
        return self.metadata.loc[name][['label','original']].values

In [ ]:
def display_images(images, cols = 5):
    plt.figure(figsize=(32,16))

    rows = len(images) // cols + 1
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)     
        
        plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
        plt.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the bottom edge are off
        labelleft=False) # labels along the bottom edge are off

## Initializing  

In [ ]:
metadata = Metadata(config.METADATA_PATH)
fake_name, real_name = multiple_face_videos[15]

# zlukluvcyp.mp4 37 sccqbzyikm.mp4
# ---
fake_path = os.path.join(config.FACE_IMAGES, fake_name)
real_path = os.path.join(config.FACE_IMAGES, real_name)

fakes = Loader(fake_path)
reals = Loader(real_path)

# --- 
idx = 6
f1 = Face(fakes[idx])
f2 = Face(reals[idx])
print(f'FAKE\tREAL\tDIFF, {np.sum(f1-f2)}')
display_images([f1.get('rgb'), f2.get('rgb'), (f1-f2).astype('uint8')], 3)

## Clustering + face diff

**Steps**
1. Extract encoding for real and fake
2. Cluster real and fake seperately
3. Relate clusters
4. Diff clusters

In [ ]:
real_encodings = [rec.get_encoding(r) for r in reals]
real_clusters = AgglomerativeClustering(n_clusters=None, distance_threshold=0.55, linkage='average').fit(real_encodings)

In [ ]:
def cluster_imgs(real_clusters):
    clustered_images = {}
    for label, p in zip(real_clusters.labels_, reals):
        if clustered_images.get(label, None) is None:
            clustered_images[label] = []

        clustered_images[label].append(Face(p))

    clustered_fake_images = {}
    for label, p in zip(real_clusters.labels_, fakes):
        if clustered_fake_images.get(label, None) is None:
            clustered_fake_images[label] = []

        clustered_fake_images[label].append(Face(p))
    
    return clustered_images, clustered_fake_images

In [ ]:
clustered_images, clustered_fake_images = cluster_imgs(real_clusters)

In [ ]:
plt.figure(figsize=(16, 8))
for cluster, imgs in clustered_images.items():
    fake_imgs = clustered_fake_images[cluster]
    difs = [np.sum(r_img - f_img) for r_img, f_img in zip(imgs, fake_imgs)]
    plt.plot(difs, label=cluster, marker='o')
    
plt.legend()
plt.title(f'Difference between cluster images fake: {fake_name} real: {real_name}')
plt.savefig(f'outputs/img_dif_{fake_name}-{real_name}.jpg')
print(fake_name, real_name)

In [ ]:
def show_clustered_imgs(clustered_images, clustered_fake_images):
    for cluster, imgs in clustered_images.items():
        print(cluster)
        display_images([i.get('rgb') for i in imgs], 18)
        display_images([i.get('rgb') for i in clustered_fake_images[cluster]], 18)

In [ ]:
for cluster, imgs in clustered_images.items():
    print(f'{cluster}: {len(imgs)}')

show_clustered_imgs(clustered_images, clustered_fake_images)

In [ ]:
# adrcjenxlz.mp4 - bad example, keep to display
# xifuovpydw.mp4

## Plotting face difference between fake and real

In [ ]:
diff_fakes = []
diff_reals = []
diff_other = []

for f, r in zip(fakes, reals):
    assert f.split('mp4_')[-1] == r.split('mp4_')[-1], f'Fake: {f}, real: {r}'
        
    fake_face = Face(f)
    real_face = Face(r)
    diff = np.sum((fake_face - real_face).astype('int')) 
    
    identifier = f.split('_')[-1]
    if identifier == '0.jpg':
        diff_reals.append(diff)
        
    elif identifier == '1.jpg':
        diff_fakes.append(diff)
    else:
        diff_other.append(diff)

plt.figure(figsize=(16,8))
plt.plot(diff_reals, label='0')
plt.plot(diff_fakes, label='1')
plt.plot(diff_other, label='other')
plt.legend()

## Analyzing face position data

In [ ]:
from utils import get_frames


class VideoReader:
    def __init__(self, name):
        self.name = name
        self.path = os.path.join(os.path.dirname(config.VIDEO_PATH), name)
        self.label = metadata[name][0]
        self.original = metadata[name][1]
        self.frames = get_frames(self.path)
        self.face_centers = []
        self.cluster_labels = None
        
        if self.label == 'REAL':
            self.coordinates = face_coordinates.loc[name].values
        else:
            self.coordinates = face_coordinates.loc[self.original].values

    def __get__(self, frame_id):
        return self.coordinates[frame_id]

    def get_face(self, frame_id, face_id=None):
        faces = []
        for i, c in enumerate(self.coordinates[frame_id]):
            if face_id is not None and i != face_id:
                continue
            faces.append(self.frames[frame_id].crop(c))
        return faces

    def get_all_faces(self):
        faces = []

        for i, frame in enumerate(self.coordinates):
            for face in frame:
                faces.append(self.frames[i].crop(face))
        return faces

    def calculate_face_centers(self):
        for frame in self.coordinates:
            for face in frame:
                self.face_centers.append([(face[0] + face[2]) / 2,
                                          (face[1] + face[3]) / 2])

    def _cluster(self, threshold=50, linkage='average'):
        self.cluster_labels = AgglomerativeClustering(n_clusters=None,
                                                distance_threshold=threshold,
                                                linkage=linkage).fit(self.face_centers).labels_

    # TODO: Add label option for fake videos
    def cluster_faces(self):
        
        clusters = {}
        self.calculate_face_centers()
        self._cluster()
        faces = self.get_all_faces()
        
        for label, face in zip(self.cluster_labels, faces):
            if clusters.get(label, None) is None:
                clusters[label] = []
            clusters[label].append(face)
        return clusters

    def play(self):
        return Video(self.path, embed=True, width=640)

In [ ]:
too_many_clusters = []

for fake_name, real_name in multiple_face_videos:
    real_video = VideoReader(real_name)
    real_clusters = real_video.cluster_faces()
    if len(real_clusters) > 4:
        too_many_clusters.append([real_name, fake_name, len(real_clusters)])
        print(real_name, fake_name, len(real_clusters))

In [ ]:
face_coordinates = pd.read_json(config.FACE_COORDINATES_PATH).T
fake_name, real_name = multiple_face_videos[7]
# fake_name = 'jjgtlwcvcd.mp4'
# real_name = metadata['jjgtlwcvcd.mp4'][1]

In [ ]:
fake_video = VideoReader(fake_name)
real_video = VideoReader(real_name)
real_video.play()

### Cluster and display

In [ ]:
# real_clusters = real_video.cluster_faces()
# fake_clusters = fake_video.cluster_faces()

fake_name, real_name, _ = too_many_clusters[2]

fake_video = VideoReader(fake_name)
real_video = VideoReader(real_name)
real_clusters = real_video.cluster_faces()
fake_clusters = fake_video.cluster_faces()

cols = max([len(l) for l in real_clusters.values()])

for k, v in real_clusters.items():
    print(k)
    display_images(v, cols=cols)
    display_images(fake_clusters[k], cols=cols)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(centers, marker='x')


In [ ]:
clustered_images, clustered_fake_images = cluster_imgs(clustered_coord)
show_clustered_imgs(clustered_images, clustered_fake_images)

In [ ]:
Video(f'D:\\DFDC\\dfdc_train_part_48/{real_name}', embed=True, width=640)

In [ ]:
Video(f'D:\\DFDC\\dfdc_train_part_48/{fake_name}', embed=True, width=640)

## Distance between face encodings

In [ ]:
for i, (f, r) in enumerate(zip(fakes, reals)):
    fake_encoding = rec.get_encoding(f)
    real_encoding = rec.get_encoding(r)
    distance = np.linalg.norm(fake_encoding - real_encoding)
    
    img1 = cv2.imread(f)
    img2 = cv2.imread(f)
    cv2.imwrite(f'test/{i}_{distance}.jpg', np.hstack((img1, img2)))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(24,8))
fake_encoding = rec.get_encoding(fakes[0])
real_encoding = rec.get_encoding(reals[0])

plt.scatter(np.arange(len(fake_encoding)), fake_encoding)
plt.scatter(np.arange(len(real_encoding)), real_encoding)

# Possible cases


1. Single cluster
     1. single face - keep all
     2. multiple faces 
     3. single face & non-face
    
    
4. Multiple clusters
    1. Single face divided
    2. Single face cluster majority + artifact - keep majority
    3. Single face cluster + equally big artifact 
    4. Same length face clusters
    5. 2 same length face clusters + smaller artifact cluster
